In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

## reading ManhattanLotLocs data into lot_df DataFrame

lot_df = pd.DataFrame(pd.read_csv('ManhattanLotLocs.txt'))

In [4]:
lot_df.head(10)

,lot_id,Block,Lot,Address,ZipCode,latitude,longitude
0,1_10,1,10,1 GOVERNORS ISLAND,10004,40.688799,-74.018675
1,1_101,1,101,1 LIBERTY ISLAND,10004,40.689920,-74.045337
2,1_201,1,201,1 ELLIS ISLAND,10004,40.698188,-74.041329
3,2_1,2,1,MARGINAL STREET,10004,40.700369,-74.012911
4,2_2,2,2,10 SOUTH STREET,10004,40.700550,-74.011588
5,2_23,2,23,1 PIER 6,10004,40.701171,-74.009243
6,3_1,3,1,10 BATTERY PARK,10280,40.702806,-74.015631
7,3_3,3,3,PETER MINUIT PLAZA,10004,40.701953,-74.013146
8,3_10,3,10,1 SOUTH STREET,10004,40.700918,-74.014444
9,4_7501,4,7501,1 WATER STREET,10004,40.702153,-74.012021


In [5]:
len(lot_df['lot_id'].unique())

42396

In [6]:
lot_df.shape ## all lot_id values are unique

(42396, 7)

In [7]:
lotdf_colindex = lot_df.columns.values
lotdf_colindex

array(['lot_id', 'Block', 'Lot', 'Address', 'ZipCode', 'latitude',
       'longitude'], dtype=object)

In [8]:
len(lot_df['Lot'].unique()) ## lot id's are composed of block_lot 

544

In [39]:
lot_id_df = lot_df.set_index(lot_df['lot_id'])
# lot_id_df.head(10)
lot_id_df = lot_id_df.drop('lot_id', 1)
lot_id_df.head()

## lot_id_df is lot_df indexed by lot_ids 

,Block,Lot,Address,ZipCode,latitude,longitude
lot_id,,,,,,
1_10,1,10,1 GOVERNORS ISLAND,10004,40.688799,-74.018675
1_101,1,101,1 LIBERTY ISLAND,10004,40.689920,-74.045337
1_201,1,201,1 ELLIS ISLAND,10004,40.698188,-74.041329
2_1,2,1,MARGINAL STREET,10004,40.700369,-74.012911
2_2,2,2,10 SOUTH STREET,10004,40.700550,-74.011588


### Calculate Manhattan Distance for each lot to the each pizza shop in miles

In [40]:
## Load Pizza Map location data (from Ksenia)

pizza_df = pd.DataFrame(pd.read_csv('pizza_locations_google.csv'))
# pizza_df.head(10)

pizza_df = pizza_df.set_index('Unnamed: 0', drop=True)

pizza_df_colindex = pizza_df.columns.values
# pizza_df_colindex

pizza_df.head()

,name,lat,lng,vicinity,open_now,rating,price_level,icon,id,photos,place_id,reference,scope,types,borough
Unnamed: 0,,,,,,,,,,,,,,,
0,P&M Classic Pizza and Restaurant,40.826154,-73.946575,"1747 Amsterdam Avenue, New York",False,4.7,1.0,https://maps.gstatic.com/mapfiles/place_api/ic...,31f44d5c640ad7d4737047373e4cf4dbd957f173,"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJqX832Xz2wokRCmDaupc-83M,CmRRAAAAOywdi6JqBTmF-VTLLJweUJOPHva0nHoXCyydOa...,GOOGLE,"['restaurant', 'food', 'point_of_interest', 'e...",NYC
1,New Broadway Pizza Coffee Shop,40.851788,-73.934708,"4312 Broadway, New York",False,4.4,2.0,https://maps.gstatic.com/mapfiles/place_api/ic...,e438e1ef8e28dfd3ebf583260d76259e238b815e,"[{'height': 4176, 'html_attributions': ['<a hr...",ChIJhx0DCh70wokRKr1DakdaiKo,CmRSAAAAriFU3PPuVv34QPIZBsAbrhzJoaURZCUDlUMNc8...,GOOGLE,"['restaurant', 'food', 'point_of_interest', 'e...",NYC
2,Tommy's Grill & Pizzeria,40.833109,-73.945095,"3750 Broadway, New York",True,3.2,1.0,https://maps.gstatic.com/mapfiles/place_api/ic...,2f47167bd8c672824a1123577e60362b74d48a6f,"[{'height': 540, 'html_attributions': ['<a hre...",ChIJrdYyy4X2wokR8AWLcUl4usA,CmRSAAAAeBpQ1xeDg9hXAA94b1hLidMs_z-lBo0uAHhPHr...,GOOGLE,"['meal_delivery', 'meal_takeaway', 'restaurant...",NYC
3,Custom Fuel Pizza,40.808704,-73.952025,"2288 Frederick Douglass Boulevard, New York",False,4.0,1.0,https://maps.gstatic.com/mapfiles/place_api/ic...,ac91f24418e1683a204b1268a0fe3b9c76874a5c,"[{'height': 750, 'html_attributions': ['<a hre...",ChIJ57jXPxP2wokRRrcwDoksk18,CmRRAAAAuG9Cxht8Cz8ge_29uFCFSXhMFpk2VMVpRkrKS3...,GOOGLE,"['bar', 'restaurant', 'food', 'point_of_intere...",NYC
4,Pizza Stop,40.815397,-73.940027,"531 Lenox Avenue, New York",True,4.0,2.0,https://maps.gstatic.com/mapfiles/place_api/ic...,72e47bb5001b6f0879bc827fe268fd0d868e5ddd,"[{'height': 419, 'html_attributions': ['<a hre...",ChIJy6G-JHj2wokRiLxKme5PyBQ,CmRRAAAAjBlEWBE8Mc2YshcH9x8ZVLvAiE-hzRqZ5ZnRGX...,GOOGLE,"['meal_delivery', 'meal_takeaway', 'restaurant...",NYC


In [11]:
## Manhattan Distance Function

## nearest listing in terms of 1) Specified Distance (user input function) 
                            ## 2) Specified Number of Pizza Shops
                            ## 3) Specified Neighborhood

In [12]:
## DataFrame: rows = lots, columns = pizza shops

In [11]:
## Creating a DataFrame to store lattitude-longitude data for Manhattan Lots
lot_indexvector = [str(k) for k in lot_id_df.index.values]
lot_lat_vector = lot_id_df['latitude']
lot_lng_vector = lot_id_df['longitude']

lot_latlng = pd.DataFrame({'latitude': lot_lat_vector, 'longitude':lot_lng_vector}, 
                          index=lot_indexvector)

lot_latlng.head()

,latitude,longitude
1_10,40.688799,-74.018675
1_101,40.689920,-74.045337
1_201,40.698188,-74.041329
2_1,40.700369,-74.012911
2_2,40.700550,-74.011588


In [12]:
## Creating a DataFrame to store lattitude-longitude data for Manhattan Pizza Shops
pizza_indexvector = pizza_df.index.values
pizza_lat_vector = pizza_df['lat']
pizza_lng_vector = pizza_df['lng']

pizza_latlng = pd.DataFrame({'latitude': pizza_lat_vector, 'longitude': pizza_lng_vector},
                           index=pizza_indexvector)

pizza_latlng['pizza_id'] = pizza_df['place_id']
pizza_latlng['pizza_id'] = [str(k) for k in pizza_latlng['pizza_id']]
pizza_latlng = pizza_latlng.set_index('pizza_id', drop=True)
pizza_latlng.head()

,latitude,longitude
pizza_id,,
ChIJqX832Xz2wokRCmDaupc-83M,40.826154,-73.946575
ChIJhx0DCh70wokRKr1DakdaiKo,40.851788,-73.934708
ChIJrdYyy4X2wokR8AWLcUl4usA,40.833109,-73.945095
ChIJ57jXPxP2wokRRrcwDoksk18,40.808704,-73.952025
ChIJy6G-JHj2wokRiLxKme5PyBQ,40.815397,-73.940027


### Manhattan Distance with Haversine Formula

Haversine Formula is used to calculate the distance of two points on a sphere given their latitudes and longitudes

a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2) 

c = 2 ⋅ asin( √a )

d = R ⋅ c

where, R = 6371 km (3959 miles) (Earth's mean radius), φ is latitude, λ is longitude. 



In [38]:
from math import sin
from math import cos
from math import atan2
from math import sqrt
from math import asin
from math import radians


def ManhattanDist(loc_lot, loc_pizza):
    '''Takes as input a lot_id and a pizza_id to calculate the distance between the
    two points with the Haversine formula. Returns a dataframe wtih rows = lot_ids, 
    columns = pizza_ids, data = distance in kms'''
    
    lot_lat = lot_latlng.loc[loc_lot]['latitude']
    lot_lng = lot_latlng.loc[loc_lot]['longitude']
    pizza_lat = pizza_latlng.loc[loc_pizza]['latitude']
    pizza_lng = pizza_latlng.loc[loc_pizza]['longitude']
    
    r = 3959
    
### If Manhattan was aligned with N-S 

#     longitude_a = sin(0)**2 + cos(radians(lot_lat))*cos(radians(lot_lat))*sin(radians((lot_lng-pizza_lng)/2))**2
    
#     latitude_a = (sin(radians(abs((lot_lat-pizza_lat)/2))))**2 + cos(radians(lot_lat))*cos(pizza_lat)*(sin(0))**2
    
#     longitude_c = c = 2*asin(sqrt(longitude_a))
                                                                              
#     latitude_c = 2*asin(sqrt(latitude_a))
                                                                              
#     d_latitude = r*latitude_c
                                                                              
#     d_longitude = r*longitude_c
                                                                              
#     d = abs(d_latitude) + abs(d_longitude)
                                                                              
#     return d 

    a = sin((radians(abs(lot_lat-pizza_lat)))/2)**2 + cos(lot_lat)*cos(pizza_lat)*sin((radians(abs(lot_lng - pizza_lng)/2)))**2

    c = 2*asin(sqrt(a))         
    
    d = r*c
    
    return d
                                                                                       
print (ManhattanDist('2_2', 'ChIJy6G-JHj2wokRiLxKme5PyBQ'))

print (ManhattanDist('3_10', 'ChIJy6G-JHj2wokRiLxKme5PyBQ'))

print (ManhattanDist('7_29', 'ChIJ57jXPxP2wokRRrcwDoksk18'))

print (ManhattanDist('3_1', 'ChIJbWfIg3j2wokRJ22QKbeFRu8'))

9.336840540537995
9.420523715895092
8.343918810496316
9.765088632893365
